In [16]:
import tensorflow as tf
import numpy as np
from trade_sim import Market
import tensorflow.contrib as contrib
import os

In [21]:
class Agent:
    def __init__(self, numberOfCurrencies, timeFrame, sess, initialPortfolio=1000):
        self._s = sess
        self.inputT = tf.placeholder(shape=[None, numberOfCurrencies, timeFrame, 3], dtype=tf.float32)
        self.conv1 = contrib.layers.conv2d(inputs=self.inputT, num_outputs=3, kernel_size=[1,3])
        self.conv2 = contrib.layers.conv2d(inputs=self.conv1, num_outputs=20, kernel_size=[1,48])
        self.conv3 = contrib.layers.conv2d(inputs=self.conv2, num_outputs=1, kernel_size=[1,1])
        self._allocate = contrib.layers.softmax(self.conv3)
        
        self.priceChanges = tf.placeholder(shape=[1,numberOfCurrencies], dtype=tf.float32)
        self.loss = (tf.reduce_sum(tf.log(tf.multiply(self._allocate, self.priceChanges))))
        self.optimizer = tf.train.AdamOptimizer(0.1)
        self._train = self.optimizer.minimize(self.loss)
        
    def act(self, observation):
        return self._s.run(self._allocate, feed_dict={self.inputT: [observation]})
    
    def train_step(self, obs, prices):
        batch_feed = {self.inputT : obs,
                     self.priceChanges: prices
                     }
        self._s.run(self._train, feed_dict=batch_feed)

In [22]:
with tf.Session() as sess:
    test1 = Agent(3, 50, sess)